# BIM relationship detection


In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# genearl
import json
import collections
import math
import uuid
import random
import pickle
import os
from itertools import islice
import numpy as np
from tqdm import tqdm

# ifc and pointcloud
import ifcopenshell
import open3d as o3d
import pymeshlab as ml
from compas.geometry import oriented_bounding_box_numpy
from scipy.spatial import distance
from ifcopenshell.util.selector import Selector

# graph 
import dgl
from dgl.data import DGLDataset
import torch

# vis
# from ipywidgets import interact
# from OCC.Core.Bnd import Bnd_Box, Bnd_OBB
# from OCC.Core.BRepBndLib import brepbndlib_AddOBB
# from OCC.Core.BRepPrimAPI import (BRepPrimAPI_MakeBox, 
# BRepPrimAPI_MakeSphere, BRepPrimAPI_MakeCylinder)
# from OCC.Core.gp import gp_Pnt, gp_XYZ, gp_Ax2, gp_Dir

# from utils.JupyterIFCRenderer import JupyterIFCRenderer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
m = ifcopenshell.open("../east_merged.ifc")
system_dict_file = "../WestDeckBox.nwd_aggregation.json"
#m = ifcopenshell.open("data/231110AC-11-Smiley-West-04-07-2007.ifc")k

In [4]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = m.by_type("IfcOwnerHistory")[0]
project = m.by_type("IfcProject")[0]
context = m.by_type("IfcGeometricRepresentationContext")[0]
floor = m.by_type("IfcBuildingStorey")[0]


## Relationship Identification

###  aggrgegation relationships

In [7]:
# flatten info dictionary
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten(el, new_key_2, sep=sep))
                else:
                    
                    items.append(el)
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten(v, new_key, sep=sep))
        else:
            print(type(v))
            items.append(v)
    return (items)


def get_systems(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        out_dict[sys_name] = flatten(sys)
        print(sys_name, len(out_dict[sys_name]))
    return out_dict
        

get_systems(system_dict_file)    
    
    

West-DeckBox-Support.rvm 57062
West-DeckBox-Electrical.rvm 32334
West-DeckBox-Structure.rvm 19892
West-DeckBox-Instrumentation.rvm 37549
West-DeckBox-Pipe.rvm 33565
West-DeckBox-Mechanical.rvm 31107
West-DeckBox-Architecture.rvm 11181
West-DeckBox-Safety.rvm 2638
West-DeckBox-HVAC.rvm 7749


{'West-DeckBox-Support.rvm': ['PVOLUME 1 of STRUCTURE /480-PSU-7168',
  'SCTN 1 of FRMWORK /480-PSU-7168/MAIN',
  'SCTN 2 of FRMWORK /480-PSU-7168/MAIN',
  'PVOLUME 1 of STRUCTURE /480-PSU-7289',
  'SCTN 1 of FRMWORK /480-PSU-7289/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /480-PSU-7289/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5777',
  'SCTN 1 of FRMWORK /481-PSU-5777/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5777/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5778',
  'SCTN 1 of FRMWORK /481-PSU-5778/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5778/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5779',
  'SCTN 1 of FRMWORK /481-PSU-5779/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5779/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5780',
  'SCTN 1 of FRMWORK /481-PSU-5780/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5780/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5781',
  '',
  'FITTING 2 of SCTN 1 of FRMWORK /481-PSU-5781/MAIN',
  'SCTN 2 of FRMWO

### Topological relationships

In [ ]:
# flatten info dictionary to branch level
def flatten_branch(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten_branch(el, new_key_2, sep=sep))
                else:
                    
                    items.append((k, el))
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten_branch(v, new_key, sep=sep))
        else:
            #print(type(v))
            items.append((k,v))
    return (items)


# get components of each branch
def get_branches(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        items = flatten_branch(sys)
        #print(items)
        d = {}
        for branch, e in items:
            if not branch in d:
                d[branch] = []
            d[branch] += [e]
        out_dict[sys_name] = d
        
        #out_dict[sys_name] = flatten_branch(sys)
        print(sys_name, len(out_dict[sys_name]))
    return out_dict
        
system = 'East-DeckBox-Piping.rvm'
branches = get_branches(system_dict_file)[system]    


In [ ]:
# draw links between connected elements, return connections
def visualize_branches(branches, ifc, draw=True, contiguous=True, dist_thresh=0.002):
    pipe_type = 'IFCPIPESEGMENT'
    fitting_type = 'IFCPIPEFITTING'

    pipe_selector = Selector()
    fitting_selector = Selector()
    pipes = pipe_selector.parse(ifc, '.' + pipe_type)
    fittings = fitting_selector.parse(ifc, '.' + fitting_type)
    fitting_names = [f.Name for f in fittings]
    pipe_names = [p.Name for p in pipes]
    print(pipes[0].Name)

    vis_dict = {}
    for k, val in branches.items():
        vis_elements = []
        connect = True
        for element in val:
            if element in pipe_names:
                vis_elements.append((element, pipe_type, connect))
                connect = True
            elif element in fitting_names:
                vis_elements.append((element, fitting_type, connect))
                connect = True
            else:
                connect = False
        vis_dict[k] = vis_elements

    error_count = 0
    count = 0
    rels = []
    selector = Selector()
    
    # enumerate through branches
    for k, val in tqdm(vis_dict.items()):
#         if count == 10:
#             break
        branch_size = len(val)
        for i, element in enumerate(val):
            #check if element is not the last element
            if (i+1) < branch_size:
                try:
                    
                    if val[i+1][2] or not contiguous:
                        rels.append([(element[0], element[1]), 
                                  (val[i+1][0], val[i+1][1])])
                        if draw:
                            element1 = selector.parse(
                                m, '.' + element[1] + '[Name *= "' + element[0] + '"]')[0]
                            element2 = selector.parse(
                                m, '.' + val[i+1][1] + '[Name *= "' + val[i+1][0] + '"]')[0]
                            draw_relationship(element[0], element1, 
                                  val[i+1][0], element2, ifc)
                
                    else:
                        element1 = selector.parse(
                            m, '.' + element[1] + '[Name *= "' + element[0] + '"]')[0]
                        element2 = selector.parse(
                            m, '.' + val[i+1][1] + '[Name *= "' + val[i+1][0] + '"]')[0]
                        
                        if element_distance(element1, element2, ifc) < dist_thresh:
                            rels.append([(element[0], element[1]), 
                                  (val[i+1][0], val[i+1][1])])
                            if draw:
                                draw_relationship(element[0], element1, 
                                  val[i+1][0], element2, ifc)
                except:
                    error_count +=1
        count +=1

    print(error_count)
    return rels


In [ ]:
rels = visualize_branches(branches, m, True)

In [ ]:
print(len(rels), rels[0])

In [ ]:
import pickle
with open('../top_rels_eastdeckbox.pkl', 'wb') as f:
    pickle.dump(rels, f)

In [ ]:
m.write('../east_vis.ifc')

## Visualization

In [ ]:
viewer = JupyterIFCRenderer(m, size=(400,300))
viewer.setAllTransparent()
viewer

### aggregation relationships

In [ ]:
picker = viewer.colorPicker()
picker


In [ ]:
out_dict = {'HVAC':['Rohrtypen:Kupfer - Hartgelötet:7718868', 'Rohrtypen:Kupfer - Hartgelötet:7718886', ],
           'electrical':[ 'Rohrtypen:Kupfer - Hartgelötet:7718872', 'Rohrtypen:Kupfer - Hartgelötet:7718880']}

# PAINT A SET OF ELEMENTS IN ONE COLOUR
def systemSelect(system):
    selector = Selector()
    for e in out_dict[system]:
        element = selector.parse(m, '.IfcProduct[Name *= "' + e + '"]')[0]
        viewer.setColor(element, picker.value)
    return system

interact(systemSelect, system=['HVAC', 'electrical'])

Instances of building elements with represenations can be selected interactivly. Information such as the attributes `GUID`, `Name` etc. are displayed to the left of the 3D viewport.

In [ ]:
# reset colours
viewer.setDefaultColors()

### topological relationships



## replace jupyter renderer

1. Compute the bounding box of ifc product directly from points
2. generate ifc elements to indicate relationships


In [ ]:
element_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element_type = "IFCPIPESEGMENT"

selector = Selector()
element = selector.parse(
    m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

shape = element.Representation.Representations[0].Items[0]
element_coords = np.array(shape.Coordinates.CoordList)
#print(element_coords)
bbox = oriented_bounding_box_numpy(element_coords)
print(bbox)

In [ ]:

# get bounding box of ifc element
def get_oriented_bbox(element):
    shape = element.Representation.Representations[0].Items[0]
    element_coords = np.array(shape.Coordinates.CoordList)
    #print(element_coords)
    bbox = oriented_bounding_box_numpy(element_coords)
    
    # identify box orientation
    l1 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[1][0], bbox[1][1], bbox[1][2]))
    l2 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[3][0], bbox[3][1], bbox[3][2]))
    l3 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[4][0], bbox[4][1], bbox[4][2]))
    half_lengths = [l1/2, l2/2, l3/2]
    
    dominant_axis = half_lengths.index(max(half_lengths))
    if dominant_axis == 0:
        dominant_direction = vector_norm([bbox[0][i] - bbox[1][i] 
                                          for i in range(3)])
    elif dominant_axis == 1:
        dominant_direction = vector_norm([bbox[0][i] - bbox[3][i] 
                                          for i in range(3)])
    else:
        dominant_direction = vector_norm([bbox[0][i] - bbox[4][i] 
                                          for i in range(3)])

    dominance_ratio = max(half_lengths)/sorted(half_lengths)[-2]
    center = [(bbox[0][i] + bbox[6][i])/2 for i in range(3)]

    #print(dominance_ratio, dominant_direction)
    #print(element_name, half_lengths, dominant_direction, center)

    #print(center)
    return([dominant_direction, max(half_lengths), 
           half_lengths, center])


# create an IFC BEAM
def CreateBeam(ifcFile, container, name, section, L, position, 
               direction, owner_history, context, colour=None):
    Z = 0.,0.,1.
    #print('length', L)
    B1 = ifcFile.createIfcBeam(create_guid(),owner_history , name)
    B1.ObjectType ='beam'
    
    #print(type(position[0]))
    B1_Point =ifcFile.createIfcCartesianPoint ( tuple(position) ) 
    #B1_Point =ifcFile.createIfcCartesianPoint ( (0.0,0.0,0.0) ) 
    B1_Axis2Placement = ifcFile.createIfcAxis2Placement3D(B1_Point)
    B1_Axis2Placement.Axis = ifcFile.createIfcDirection(direction)
    B1_Axis2Placement.RefDirection =ifcFile.createIfcDirection(
        np.cross(direction, Z).tolist())

    B1_Placement = ifcFile.createIfcLocalPlacement(
        container.ObjectPlacement,B1_Axis2Placement)
    B1.ObjectPlacement=B1_Placement
    B1_ExtrudePlacement = ifcFile.createIfcAxis2Placement3D(ifcFile.createIfcCartesianPoint ( (0.,0.,0.) )   )
   
    B1_Extruded=ifcFile.createIfcExtrudedAreaSolid()
    B1_Extruded.SweptArea=section
    B1_Extruded.Position=B1_ExtrudePlacement
    B1_Extruded.ExtrudedDirection = ifcFile.createIfcDirection(Z)
    B1_Extruded.Depth = L
    
    # add colour
    if colour is not None:
        shade = ifc.createIfcSurfaceStyleRendering(colour)
        surfaceStyle = ifc.createIfcSurfaceStyle(colour.Name, "BOTH",(shade,))
        presStyleAssign = ifc.createIfcPresentationStyleAssignment((surfaceStyle,))
        ifcFile.createIfcStyledItem(B1_Extruded, (presStyleAssign,), colour.Name)


    B1_Repr=ifcFile.createIfcShapeRepresentation()
    B1_Repr.ContextOfItems=context
    B1_Repr.RepresentationIdentifier = 'Body'
    B1_Repr.RepresentationType = 'SweptSolid'
    B1_Repr.Items = [B1_Extruded]
    
    B1_DefShape=ifcFile.createIfcProductDefinitionShape()
    B1_DefShape.Representations=[B1_Repr]
    B1.Representation=B1_DefShape
    
    Flr1_Container = ifcFile.createIfcRelContainedInSpatialStructure(create_guid(),owner_history)
    Flr1_Container.RelatedElements=[B1]
    Flr1_Container.RelatingStructure= container
    
    
def Circle_Section(r, ifcfile):
    B1_Axis2Placement2D =ifcfile.createIfcAxis2Placement2D( 
                          ifcfile.createIfcCartesianPoint( (0.,0.,0.) ) )
    B1_AreaProfile = ifcfile.createIfcCircleProfileDef("AREA")
    B1_AreaProfile.Position = B1_Axis2Placement2D 
    B1_AreaProfile.Radius = r
    return B1_AreaProfile


#get_oriented_bbox(element_name, element_type)

In [ ]:

def get_point_along_axis(init_point, axis, half_length, edge_distance):
    #print(half_length)
    return (init_point + axis*(half_length - edge_distance))


def sq_distance(x1, y1, z1, x2, y2, z2):
    return ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)


def vector_norm(vec):
    den = math.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    return [vec[0]/den, vec[1]/den, vec[2]/den]


# identify a candidate for an edge of the relationship visualization
def get_corner(obb, center_other, edge_distance):
    direction = obb[0]
    center = obb[3]

    candidate1 = [get_point_along_axis(center[i], direction[i], 
                                    obb[1], edge_distance) 
               for i in range(3)]
    candidate2 = [get_point_along_axis(center[i], direction[i], 
                                    -1*obb[1], -1*edge_distance) 
               for i in range(3)]
    # print(corner1, corner2)
    
    dist1 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate1[0],
                   candidate1[1], candidate1[2])    
    dist2 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate2[0],
                   candidate2[1], candidate2[2]) 
    
    if dist1 < dist2:
        return candidate1
    else:
        return candidate2

    
# get points belonging to an ifc element
def get_points(element, ifc):
    shape = element.Representation.Representations[0].Items[0]
    return (np.array(shape.Coordinates.CoordList))   
    
    
# calculate min distance between two ifc elements
def element_distance(element1, element2, ifc):
    points1 = get_points(element1, ifc)
    points2 = get_points(element2, ifc)
    return np.min(distance.cdist(points1, points2, 'sqeuclidean'))


# e1 = 'ELBOW 2 of BRANCH /AU-8110202-WD-MDA-02/B1'
# e1t = 'IFCPIPEFITTING'
# e2 = 'TUBE 1 of BRANCH /AU-8110202-WD-MDA-02/B1'
# e2t = 'IFCPIPESEGMENT'
# element_distance(e1, e1t, e2, e2t, m)
    
def draw_sphere(point, radius, colour, viewer):
    point = gp_Pnt(point)
    ball = BRepPrimAPI_MakeSphere(point, radius).Shape()
    viewer.DisplayShape(ball, shape_color = colour, 
                        transparency=True, opacity=0.8)


def draw_cylinder(p1, p2, radius, colour,  element_name1, element_name2, ifc):
    sectionC1 = Circle_Section(r=radius, ifcfile=ifc)
    name='rel '+element_name1 + ' x ' + element_name2
    ConnectingBeam_1 = CreateBeam(ifc, container=floor, name=name, 
                                  section= sectionC1, 
                                  L=math.sqrt(sq_distance(p1[0],p1[1],p1[2],p2[0],p2[1],p2[2])),
                                  position=([p2[0].item(), p2[1].item(), p2[2].item()]),
                                  direction=((p1[0] - p2[0]).item(), (p1[1] - p2[1]).item(), 
                                             (p1[2] - p2[2]).item()),
                                  owner_history=owner_history, context=context, colour=colour)
#     ifc.create_entity('IfcPipeSegment', GlobalId=ifcopenshell.guid.new(), 
#                       Name='rel '+element_name1 + ' x ' + element_name2)
    
# def draw_cylinder(p1, p2, radius, colour, viewer):
#     x = (p1.X() + p2.X())/2
#     y = (p1.Y() + p2.Y())/2
#     z = (p1.Z() + p2.Z())/2
#     p = gp_Pnt(p2.X(), p2.Y(), p2.Z())
#     v = gp_Dir((p1.X() - p2.X()), (p1.Y() - p2.Y()), (p1.Z() - p2.Z()))
#     ax = gp_Ax2(p, v)
#     h = math.sqrt(sq_distance(p1.X(), p1.Y(), p2.Z(), 
#                               p2.X(), p2.Y(), p2.Z()))
#     print(x,y,z,p,v,ax,h)
    
#     cylinder = BRepPrimAPI_MakeCylinder(ax, radius, h).Shape()
#     viewer.DisplayShape(cylinder, shape_color = colour, 
#                         transparency=True, opacity=0.8)
    

# draw a visual indication of a topological relationship    
def draw_relationship (element_name1, element1, element_name2,
                       element2, ifc, colour=None):
    # define params
    #radius = 0.03
    radius_expansion = 1.3
    threshold = 0.1
    edge_distance = 0.1
    
    # get bboxes
    obb1 = get_oriented_bbox(element1)
    obb2 = get_oriented_bbox(element2)
    
    # if bboxes are roughly square, then use centerpoint, 
    # otherwise use a point closer to the edge
    if obb1[1] < threshold:
        corner1 = obb1[3]
    else:
        corner1 = get_corner(obb1, obb2[3], edge_distance)
    if  obb2[1] < threshold:
        corner2 = obb2[3]
    else:
        corner2 = get_corner(obb2, obb1[3], edge_distance)
    #print(corner1.X(),corner1.Y(),corner1.Z(),corner2.X(),corner2.Y(),corner1.Z())
    #print('rel', corner1, corner2, element_name1, element_name2)
    
    #dynamically scale radius
    radius = max(min(obb1[2]), min(obb2[2])) * radius_expansion
    
    draw_cylinder(corner1, corner2, radius, colour, 
                  element_name1, element_name2, ifc)
#     draw_sphere(corner1, radius, colour, viewer)
#     draw_sphere(corner2, radius, 'red', viewer)
 

In [ ]:
# element1_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element1_type = "IFCPIPESEGMENT"
# element2_name = "ELBOW 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element2_type = "IFCPIPEFITTING"

# element3_name = "TUBE 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element3_type = "IFCPIPESEGMENT"
# element4_name = "ELBOW 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element4_type = "IFCPIPEFITTING"
# element5_name = "TUBE 3 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element5_type = "IFCPIPESEGMENT"

# draw_relationship(element1_name, element1_type, 
#                   element2_name, element2_type, m)
# draw_relationship(element2_name, element2_type, 
#                   element3_name, element3_type, m)
# draw_relationship(element3_name, element3_type, 
#                   element4_name, element4_type, m)
# draw_relationship(element4_name, element4_type, 
#                   element5_name, element5_type, m)
#element1_center, element1_coords = get_element_deets()



# centerpoint =gp_Pnt(element1_center)
# ball = BRepPrimAPI_MakeSphere(centerpoint, 0.02).Shape()

# Misc

In [ ]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)
pdct_shape = ifcopenshell.geom.create_shape(settings, inst=element)
viewer.DisplayShape(ball, shape_color = 'blue', transparency=False, opacity=0.5)

In [ ]:
o_bbox1 = get_oriented_boundingbox(list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element1)])
bbox_center1 = [o_bbox[0].X(), o_bbox[0].Y(), o_bbox[0].Z()]


In [ ]:
viewer.shapedict.values()

In [ ]:
list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]

In [ ]:
viewer._shapes

In [ ]:
BoundingBox([[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]]).xmin

In [ ]:
[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]

In [ ]:
print(len(element.Representation.Representations[0].Items[0].CoordIndex))

In [ ]:
element.Representation.Representations[0].Items[0].CoordIndex = element.Representation.Representations[0].Items[0].CoordIndex[:1]

## Selecting instances 

The instance object currently selected in the 3D view can be assigned to a avariable using the `.getSelectedProduct()` method. 

Provding the plain varialbe name call sthe `Display()`-method of the instance and prints the line of the original ifc-Datei the  [SPFF-format](https://en.wikipedia.org/wiki/ISO_10303-21).


In [ ]:
selection = viewer.getSelectedProduct()
selection

## Show / Hide Objects
Programatically or interactively selected projects can be hidden or shown in the viewer using `setVisible()`.


In [ ]:
viewer.setVisible(selection, False)
viewer

Opening elements are displayed by default. Let's hide them.

In [ ]:
for opening in m.by_type("IfcOpeningElement"):
    viewer.setVisible(opening, False)

## Coloring Objects
Objects can be colored either by hexadicimal numbers provided as strings as they are common in e.g. in HTML and CSS and can be retrieved from many ressources

In [ ]:

viewer.setColorSelected("#ddffaa")
viewer

If you do not happen to have a color in mind  just call a color picker from the viewer:

In [ ]:
picker = viewer.colorPicker()
picker

In [ ]:
viewer.setColorSelected(picker.value)

## IFC to cloud

sample points from ifc model

In [ ]:
ifc = ifcopenshell.open("../east_merged.ifc")

In [ ]:
element_type = 'IFCPIPESEGMENT'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(len(tees))

In [ ]:
# pick a point uniformly in a triangle
def uniform_triangle(u, v):
    while True:
        s = random.random()
        t = random.random()
        in_triangle = s + t <= 1
        p = s * u + t * v if in_triangle else (1 - s) * u + (1 - t) * v
        yield p


# uniformly sample points within triangle
def sample_points(p, q, r, n):   
    it = uniform_triangle((q - p), (r - p))
    points = np.array(list(islice(it, 0, n)))
    points += p
    return points


# convert an element to a point cloud
def element_to_cloud(element, save_path=None, density=0):
    shape = element.Representation.Representations[0].Items[0]
    boundaries = np.array(shape.Coordinates.CoordList)
    
    # determine sampling target 
    point_count = len(boundaries)
    samples = 10 if density == 0 else math.ceil(density * 6 / point_count)
        
    # get additional points by sampling from mesh triangles
    limit = point_count -2
    #print (limit)
    centroids = []
    for j in range(0, point_count, 3):
        if j < limit:
            # centroids.append([(boundaries[j][k] + boundaries[j+1][k] 
            # + boundaries[j+2][k])/3 for k in range(3)])
            centroids.extend(sample_points((boundaries[j]), (boundaries[j+1]),
                                           (boundaries[j+2]), samples))
    boundaries = np.concatenate([boundaries, np.array(centroids)])
    #print(len(boundaries))
    
    # downsample to fixed length
    if density > 0:
        boundaries = boundaries[np.random.choice(boundaries.shape[0], density, 
                                                 replace=False), :]
    
    # convert to pointcloud
    if save_path is not None:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(boundaries)
        o3d.io.write_point_cloud(save_path, pcd)
    return (boundaries)

In [ ]:
for i, element in tqdm(enumerate(tees)):
    save_path = "../east_tee_clouds/" + str(i) + ".ply"
    cloud = element_to_cloud(element, save_path, 1000)

## Graph dataset

### get nodes & edges

In [ ]:
# get center, dimensions and direction of ifc element
def get_dimensions (element):
    shape = element.Representation.Representations[0].Items[0]
    element_coords = np.array(shape.Coordinates.CoordList)
    
    bbox = oriented_bounding_box_numpy(element_coords)
    center = [(bbox[0][i] + bbox[6][i])/2 for i in range(3)]
    
    # identify box orientation
    l1 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[1][0], bbox[1][1], bbox[1][2]))
    l2 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[3][0], bbox[3][1], bbox[3][2]))
    l3 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[4][0], bbox[4][1], bbox[4][2]))
    lengths = [l1, l2, l3]
    
    dominant_axis = lengths.index(max(lengths))
    if dominant_axis == 0:
        dominant_direction = vector_norm([bbox[0][i] - bbox[1][i] 
                                          for i in range(3)])
    elif dominant_axis == 1:
        dominant_direction = vector_norm([bbox[0][i] - bbox[3][i] 
                                          for i in range(3)])
    else:
        dominant_direction = vector_norm([bbox[0][i] - bbox[4][i] 
                                          for i in range(3)])
        
    return(center, lengths, dominant_direction)


# get node info
def process_nodes(ifc, types):
    # load elements
    element_type1 = 'IFCPIPESEGMENT'
    element_type2 = 'IFCPIPEFITTING'
    selector = Selector()
    segments = selector.parse(ifc, '.' + element_type1)
    fittings = selector.parse(ifc, '.' + element_type2)
    elements = segments + fittings
    
    # generate node features
    nodes = []
    points = []
    error_count = 0
    for i, el in tqdm(enumerate(elements)):
        try:
            center, lengths, dominant_direction = get_dimensions(el)
            
            # find element type
            found = False
            for j, t in enumerate(types):
                if t in el.Name:
                    el_type = j
                    found = True
                    break
            if not found:
                el_type = len(types)

            element_points = element_to_cloud(el, None, 1000)
            node = [el_type, center, lengths, dominant_direction, 
                          el.id()]
            
            nodes.append(node)
            points.append(element_points)
            
        except Exception as e:
            error_count += 1
            print(el.Name)
            print(e)
    
    print(error_count)
    return([nodes, points])


# derive edges from node and relationship information
def process_edges(ifc, nodes, rels):
    selector = Selector()
    pipe_type = 'IFCPIPESEGMENT'
    fitting_type = 'IFCPIPEFITTING'

    pipe_selector = Selector()
    fitting_selector = Selector()
    pipes = pipe_selector.parse(ifc, '.' + pipe_type)
    fittings = fitting_selector.parse(ifc, '.' + fitting_type)
    elements = pipes + fittings
    element_names = [e.Name for e in elements]
    print(len(element_names))
    edges = []
    error_count = 0
    
    # lookup matching index from nodes
    for rel in tqdm(rels):
        element1 = element_names.index(rel[0][0])
        element1_id = elements[element1].id() 
        element2= element_names.index(rel[1][0])
        element2_id = elements[element2].id()
        
        element1_found, element2_found = False, False
        for i, node in enumerate(nodes):
            if element1_id == node[4]:
                element1_index = i
                element1_found = True
            if element2_id == node[4]:
                element2_index = i
                element2_found = True
            if (element1_found and element2_found):
                edges.append([element1_index, element2_index])
                break
        if not (element1_found and element2_found):
            #print(rel[0][0], rel[1][0])
            error_count += 1
                        
    print(error_count, len(edges))
    return (edges)


In [ ]:
types = ['FLANGE', 'ELBOW', 'TEE', 'TUBE', 'BEND']
node_info = process_nodes(ifc, types)

In [ ]:
print(len(node_info), len(node_info[0]), len(node_info[1]))

with open('../nodes_eastdeckbox.pkl', 'wb') as f:
    pickle.dump(node_info, f)

In [ ]:
with open('../top_rels_eastdeckbox.pkl', 'rb') as f:
    rels = pickle.load(f)
with open('../nodes_eastdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    
edges = process_edges(ifc, nodes, rels)

In [ ]:
with open('../edges_eastdeckbox.pkl', 'wb') as f:
    pickle.dump(edges, f)

In [ ]:
# sanity checks

# print(points.shape, labels.shape, centers.shape, lengths.shape, directions.shape)
# print(points[0][0], labels[0], centers[0], lengths[0], directions[0])
# print(5 in labels)

# edges_src = edges[:,0]
# edges_dst = edges[:,1]
# print(edges_src)
# print(np.max(edges_dst))

### create dataset

In [ ]:
# define industrial facility graph dataset
class IndustrialFacilityDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='industrial_facility')

    def process(self):
        # data loading
        with open('../nodes_westdeckbox.pkl', 'rb') as f:
            node_info = pickle.load(f)
        with open('../edges_westdeckbox.pkl', 'rb') as f:
            edges = pickle.load(f)
        
        # node features
        points = torch.from_numpy(np.array(node_info[1]))
        labels = torch.from_numpy(np.array([i[0] for i in node_info[0]]))
        centers = torch.from_numpy(np.array([i[1] for i in node_info[0]]))
        lengths = torch.from_numpy(np.array([i[2] for i in node_info[0]]))
        directions = torch.from_numpy(np.array([i[3] for i in node_info[0]]))
        
        # edges
        edges = np.array(edges)
        edges_src = torch.from_numpy(edges[:,0])
        edges_dst = torch.from_numpy(edges[:,1])

        # create graph
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes = len(node_info[0]))
        self.graph.ndata['centers'] = centers
        self.graph.ndata['directions'] = directions
        self.graph.ndata['lengths'] = lengths
        self.graph.ndata['label'] = labels
        #self.graph.ndata['points'] = points
        #self.graph.edata['weight'] = edge_features

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
#         n_nodes = nodes_data.shape[0]
#         n_train = int(n_nodes * 0.6)
#         n_val = int(n_nodes * 0.2)
#         train_mask = torch.zeros(n_nodes, dtype=torch.bool)
#         val_mask = torch.zeros(n_nodes, dtype=torch.bool)
#         test_mask = torch.zeros(n_nodes, dtype=torch.bool)
#         train_mask[:n_train] = True
#         val_mask[n_train:n_train + n_val] = True
#         test_mask[n_train + n_val:] = True
#         self.graph.ndata['train_mask'] = train_mask
#         self.graph.ndata['val_mask'] = val_mask
#         self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1


In [ ]:
dataset = IndustrialFacilityDataset()
graph = dataset[0]

print(graph)

## Evaluate GNN

In [3]:
# load edges
with open('../eval/pos_edges.pkl', 'rb') as f:
            u, v = pickle.load(f)
with open('../eval/neg_edges.pkl', 'rb') as f:
            neg_u_full,neg_v_full = pickle.load(f)
print(len(neg_u_full))


283235946


In [4]:
# load predicted scores
with open('../eval/pos_score.pkl', 'rb') as f:
            pos_score = pickle.load(f)
with open('../eval/neg_score.pkl', 'rb') as f:
            neg_score= pickle.load(f)
print(len(neg_score))

283235946


In [5]:
print(len(neg_score), len(pos_score))

283235946 29784


In [11]:
print(type(neg_score))

<class 'numpy.ndarray'>


In [6]:
# check metrics
threshold = 0.5
FNs, TPs, FPs, TNs = [], [], [], []

# TPs / FNs
for i, score in enumerate(pos_score):
    sig = 1/(1 + np.exp(-score))
    if sig > threshold:
        TPs.append([u[i].item(), v[i].item()])
    else:
        FNs.append([u[i].item(), v[i].item()])

# FPs
for i, score in tqdm(enumerate(neg_score)):
    sig = 1/(1 + np.exp(-score))
    if sig > threshold:
        FPs.append([neg_u_full[i], neg_v_full[i]])

print(len(TPs), len(FPs), len(FNs) )
TPs = np.array(TPs)
FPs = np.array(FPs)
FNs = np.array(FNs)

283235946it [07:35, 621429.90it/s]


26011 13267667 3773


In [8]:
print(1/(1 + np.exp(-neg_score[0])))

3.268730827463247e-103


In [9]:
with open('../eval/metrics.pkl', 'wb') as f:
    pickle.dump([TPs, FPs, FNs], f)

In [6]:
with open('../eval/metrics.pkl', 'rb') as f:
    TPs, FPs, FNs = pickle.load(f)

In [7]:
precision = len(TPs)/(len(TPs)+len(FPs))
recall = len(TPs)/(len(TPs)+len(FNs))
accuracy = (len(TPs)-len(FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

0.0019566443538048687 0.8733212463067419 0.9531484447483287


In [18]:
site = 'east'
data_path = '../'
node_file = "nodes_" + site + "deckbox.pkl"
with open(data_path + node_file, 'rb') as f:
    node_info = pickle.load(f)

In [11]:
# check if positive predictions fall within distance threshold
def check_predictions(preds, point_info, dist_thresh=0.002):
    refined_preds = []
    
    for pair in tqdm(preds):
        dist = np.min(distance.cdist(
            point_info[pair[0]], point_info[pair[1]], 'sqeuclidean'))
        if (dist < dist_thresh):
            refined_preds.append(pair)
    return refined_preds

In [12]:
refined_TPs = check_predictions(TPs, node_info[1])
refined_FPs = check_predictions(FPs, node_info[1])

print(len(refined_TPs), len(refined_FPs))

100%|████████████████████████████████████████████████████████████████████| 13267667/13267667 [8:19:54<00:00, 442.33it/s]


26003 25392


In [13]:
with open('../eval/refined.pkl', 'wb') as f:
    pickle.dump([refined_TPs, refined_FPs], f)

In [8]:
with open('../eval/refined.pkl', 'rb') as f:
    refined_TPs, refined_FPs = pickle.load(f)
print(len(refined_TPs))

26003


In [9]:
precision = len(refined_TPs)/(len(refined_TPs)+len(refined_FPs))
recall = len(refined_TPs)/(len(refined_TPs)+len(FNs))
accuracy = (len(refined_TPs)-len(refined_FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

0.5059441579920225 0.8732872111767866 0.999897011897627


In [ ]:
# pos_sig = 1/(1 + np.exp(-pos_score))
# neg_sig = 1/(1 + np.exp(-neg_score))


In [5]:
pos_score= pos_score[:int(len(pos_score)/10)]
neg_score= pos_score[:int(len(neg_score)/10)]

In [6]:
len(neg_score)

2978

In [27]:
scores = np.concatenate([pos_score, neg_score])

In [ ]:
sig = 1/(1 + np.exp(-scores))

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

RocCurveDisplay.from_predictions(labels, scores)
plt.show()

### visualize predictions

In [ ]:
ifc = ifcopenshell.open("../east_merged.ifc")

In [ ]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = ifc.by_type("IfcOwnerHistory")[0]
project = ifc.by_type("IfcProject")[0]
context = ifc.by_type("IfcGeometricRepresentationContext")[0]
floor = ifc.by_type("IfcBuildingStorey")[0]


In [ ]:
red = ifc.createIfcColourRgb('red', Red=0.9, Green=0.0, Blue=0.0)
green = ifc.createIfcColourRgb('green', Red=0.0, Green=0.9, Blue=0.0)
yellow = ifc.createIfcColourRgb('yellow', Red=0.9, Green=0.9, Blue=0.0)

In [ ]:
# visualize results on ifc file
def draw_predictions(preds, nodes, ifc, colour):
    for pair in tqdm(preds):
        element1 = ifc.by_id(nodes[pair[0]][4])
        element1_name = element1.Name
        element2 = ifc.by_id(nodes[pair[1]][4])
        element2_name = element2.Name
        
        draw_relationship(element1_name, element1, 
                                      element2_name, element2, ifc, colour)

In [ ]:
draw_predictions(refined_TPs, node_info[0], ifc, green)

In [ ]:
draw_predictions(refined_FPs, node_info[0], ifc, yellow)

In [ ]:
draw_predictions(FNs, node_info[0], ifc, red)

In [ ]:
ifc.write('../eval/pred_vis.ifc')

### cleanup predictions

In [10]:
def remove_repetitions(preds):
    non_rep = []
    for i, pair in enumerate(preds):
        found = False
        for j, pair2 in enumerate(preds[i:]):
            if pair[0] == pair2[1] and pair[1] == pair2[0]:
                found = True
                #break
        if not found:
            non_rep.append(pair)
    
    return non_rep

In [11]:
def compare_preds(preds1, preds2):
    for i in range(len(preds1)):
        preds1[i].sort()
    for i in range(len(preds2)):
        preds2[i].sort()
    non_rep = []
    
    for i, pair in enumerate(preds1):
        found = False
        for j, pair2 in enumerate(preds2):
            if pair[0] == pair2[0] and pair[1] == pair2[1]:
                found = True
                break
        if not found:
            non_rep.append(pair)
    
    return non_rep

In [12]:
print(len(refined_TPs))
non_rep_TPs = remove_repetitions(refined_TPs)
print(len(non_rep_TPs))

26003
13181


In [13]:
print(len(refined_FPs))
non_rep_FPs = remove_repetitions(refined_FPs)
print(len(non_rep_FPs))

25392
13071


In [14]:
print(len(FNs))
non_rep_FNs = remove_repetitions(FNs)
print(len(non_rep_FNs))

3773
2066


In [15]:
refined_FNs = compare_preds(non_rep_FNs, non_rep_TPs)
print(len(non_rep_FNs), len(refined_FNs))

2066 1707


In [16]:
precision = len(non_rep_TPs)/(len(non_rep_TPs)+len(non_rep_FPs))
recall = len(non_rep_TPs)/(len(non_rep_TPs)+len(refined_FNs))
accuracy = (len(non_rep_TPs)-len(refined_FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

0.5020950784702118 0.8853439011284255 0.9998517469797705


In [24]:
# element type wise precision recall analysis
def sort_type(preds, nodes):
    bins = np.zeros([4,4])
    for p in preds:
        x=nodes[p[0]][0]
        y= nodes[p[1]][0]
        if x == 4:
          x = 3
        if y == 4:
          y = 3
        li = [x,y]
        li.sort()
        x,y = li[0],li[1]
        
        bins[x][y] += 1
    return bins

In [27]:
tp_bins = sort_type(non_rep_TPs,node_info[0] )
fp_bins = sort_type(non_rep_FPs,node_info[0] )
fn_bins = sort_type(refined_FNs,node_info[0] )

In [26]:
print(tp_bins)

[[ 252.  250.   44. 1172.]
 [   0.  170.   73. 6524.]
 [   0.    0.   17.  755.]
 [   0.    0.    0. 3924.]]


In [28]:
recall = tp_bins/(tp_bins+fn_bins)
precision = tp_bins/(tp_bins+fp_bins)

In [31]:
print(precision)
print(recall)

[[0.74117647 0.93984962 0.88       0.93089754]
 [       nan 0.89005236 0.60330579 0.96494601]
 [       nan        nan 0.94444444 0.71293673]
 [       nan        nan        nan 0.24241675]]
[[1.         0.9765625  1.         0.97342193]
 [       nan 0.97142857 0.97333333 0.94427558]
 [       nan        nan 1.         0.93556382]
 [       nan        nan        nan 0.76208973]]


In [23]:
#print((2*precision*recall)/(precision+recall))

0.6407875546912981


[  59 1440]


### Analyse dataset

In [ ]:
# analyse dataset

# load data
site = 'east'

def analyse_dataset(site):
  data_path = "/content/drive/MyDrive/graph/"
  edge_file = "edges_" + site + "deckbox.pkl"
  node_file = "nodes_" + site + "deckbox.pkl"
  with open(data_path + node_file, 'rb') as f:
      node_info = pickle.load(f)
  with open(data_path + edge_file, 'rb') as f:
      edges = pickle.load(f)

  # get element type counts
  labels = np.array([i[0] for i in node_info[0]])
  unique, counts = np.unique(labels, return_counts=True)
  print(dict(zip(unique, counts)))

  # get connection counts
  counts = np.zeros((5,5), dtype=int)

  for edge in edges:
    x = labels[edge[0]]
    y = labels[edge[1]]
    if x == 4:
      x = 3
    if y == 4:
      y = 3
    li = [x,y]
    li.sort()
    x,y = li[0],li[1]
    counts[x][y] += 1

  return(counts)

c_east = analyse_dataset('east')
c_west = analyse_dataset('west')
count = c_east + c_west
print(count)

<br><br><br><br>

### Downsampling



In [ ]:
ifc = ifcopenshell.open("../deckboxtee.ifc")

element_type = 'IFCPIPEFITTING'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(tees[0])

In [ ]:
for element in tqdm(tees):
    # create pymeshlab mesh
    #print(element)
    shape = element.Representation.Representations[0].Items[0]
    element_faces = [np.array([i[0]-1, i[1]-1, i[2]-1]) for i in shape.CoordIndex]
    element_coords = np.array(shape.Coordinates.CoordList)
    print(len(element_faces))
#     mesh = ml.Mesh(element_coords, element_faces)
#     ms = ml.MeshSet()
#     ms.add_mesh(mesh, "x")
#     #ms.save_current_mesh("../output.ply")
    
#     # downsample and reassign
#     ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(2))
#     m = ms.current_mesh()
#     shape.CoordIndex = m.face_matrix().tolist()
#     shape.Coordinates.CoordList = m.vertex_matrix().tolist()
    

In [ ]:
ifc.write('../added.ifc') 

In [ ]:
viewer = JupyterIFCRenderer(ifc, size=(400,300))
viewer

In [ ]:
## meshlab downsampling -DOESNT WORK

In [ ]:

print('input mesh has', mesh.vertex_number(), 'vertex and', mesh.face_number(), 'faces')


#Estimate number of faces to have 100+10000 vertex using Euler

#Simplify the mesh. Only first simplification will be agressive
#ms.apply_filter('simplification_quadric_edge_collapse_decimation', targetfacenum=numFaces, preservenormal=False)
ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(5))
print("Decimated to", numFaces, "faces mesh has", ms.current_mesh().vertex_number(), "vertex")


m = ms.current_mesh()
print('output mesh has', m.vertex_number(), 'vertex and', m.face_number(), 'faces')
ms.save_current_mesh('../output.ply')

In [ ]:
print(m.face_matrix())

In [ ]:
shape.CoordIndex = m.face_matrix().tolist()
shape.Coordinates.CoordList = m.vertex_matrix().tolist()